# Telegram Sentiment Analysis

In [1]:
# Importing basic libraries
import pandas as pd
import numpy as np
import time as t
from configparser import ConfigParser
config = ConfigParser()
import asyncio
import nest_asyncio
nest_asyncio.apply()

# Plotting libraries
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(rc={'figure.figsize':(12,10)})
# sns.set_style('white')
plt.style.use('fivethirtyeight')

from tqdm import tqdm
tqdm.pandas()

# Sentiment Analysis Library
import telegramscraping
import datapreprocessing
import sentimentscoring

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
# Setting configuration values
config.read('/Users/pauldoan/Documents/Token Metrics/1 Admin/telegram_credentials.ini')

api_id = config['Telegram']['api_id']
api_hash = config['Telegram']['api_hash']
phone = config['Telegram']['phone']
username = config['Telegram']['username']

from telethon import TelegramClient
from telethon.errors import SessionPasswordNeededError

# Create the client and connect
client = TelegramClient(username, api_id, api_hash)
await client.start()
print("Client successfully created.")
# Ensure you're authorized
if not await client.is_user_authorized():
    client.send_code_request(phone)
    try:
        client.sign_in(phone, input('Enter the code: '))
    except SessionPasswordNeededError:
        client.sign_in(password=input('Password: '))

Client successfully created.


## Retrieving telegram group id from Token Metrics db

In [3]:
channel = telegramscraping.find_tm_group('Chainlink')

Telegram group found: chainlinkofficial


## Getting channel messages


In [6]:
data = asyncio.run(telegramscraping.scrape_telegram_messages(channel, client, limit_msgs=1000, timespan=None))

Getting messages data for telegram group: chainlinkofficial ...
Date limit reached.
Done.



In [ ]:
data = datapreprocessing.preprocess_telegram(data)
data.head()

## Importing prices form Token Metrics db

In [ ]:
# Importing price data for the coin
##########################
get_price_info = True
coin_symbol = 'LINK'
##########################

if get_price_info:
    prices = sentimentscoring.get_price_data(coin_symbol)

### Sentiment

In [ ]:
# Extracting sentiment of tweets
sentiment_data = sentimentscoring.get_sentiment(data)

### Visualization of some tweets

In [ ]:
########################
nb_to_print = 10
########################

print('Worst messages:\n')
sentimentscoring.print_worse(sentiment_data, nb_to_print)
print('\n--------\nBest messages:\n')
sentimentscoring.print_best(sentiment_data, nb_to_print)

### Visualization - Simple moving average

In [ ]:
sentimentscoring.plot_moving_average(sentiment_data, price_data=prices, window1=7, window2=15, type='simple')

### Visualization - Weighted Moving Average

In [ ]:
sentimentscoring.plot_moving_average(sentiment_data, price_data=prices, window1=7, window2=15, type='weighted')

### Visualization - Exponential Moving Average

In [ ]:
sentimentscoring.plot_moving_average(sentiment_data, price_data=prices, window1=7, window2=15, type='exp')

## MACD

In [ ]:
sentimentscoring.plot_moving_average(sentiment_data, price_data=prices, window1=12, window2=26, type='macd')

In [ ]:
sentimentscoring.plot_moving_average(sentiment_data, window1=12, window2=26, type='macd')